<a href="https://www.kaggle.com/code/barisulusoy/r-n-yorumlar-duygu-analizi?scriptVersionId=99817327" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Giriş 
Bu araştırmada, birkaç e-ticaret sitesinden toplanmış ürün yorumları ve yıldızlarından oluşan veri seti üzerinde duygu analizi çalışması yapılmıştır.

<font color = "blue">
    İçerik:
    
   1. [Veri Yüklenmesi](#1)
   2. [Veri Analizi](#2)
   3. [Yorumların Düzenlenmesi (Metin Düzenleme)](#3)
   4. [Kelime Kökleme](#4)
   5. [TF-IDF](#5)
   6. [Duygu Analizi](#6)
   7. [Yeni Gelen Yorum Duygusu Tahmini](#7)

<a id="1"></a><br>
# Veri Yüklenmesi

In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


df = pd.read_csv("../input/eticaret-urun-yorumlari/e-ticaret_urun_yorumlari.csv",delimiter=';')
df

,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1
...,...,...
15165,ışık seviyesi rezalet,0
15166,Hic begenmedim. Aydinlatma hic yok ve her kapa...,0
15167,2 gün sonra hoparlörü bozuldu kullanışsız,0
15168,aşırı boğuk bir sesi ve rengi var kumanda heme...,0


<a id="2"></a><br>
# Veri Analizi

In [2]:
# boş veri kontrolü
df.isna().value_counts()

Metin  Durum
False  False    15170
dtype: int64

In [3]:
# sınıf dağılımı (0: olumsuz , 1: olumlu , 2: nötr)
df["Durum"].value_counts()

0    6978
1    6799
2    1393
Name: Durum, dtype: int64

Boş verimiz yok fakat sınıflarımız arasında dengesiz bir dağılım bulunmaktadır.

<a id="3"></a><br>
# Yorumların Düzenlenmesi (Metin Düzenleme)

In [4]:
df['Yeni Metin'] = df['Metin']
#buyuk-kucuk donusumu
df['Yeni Metin'] = df['Yeni Metin'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#noktalama işaretleri
df['Yeni Metin'] = df['Yeni Metin'].str.replace('[^\w\s]','')
#sayılar
df['Yeni Metin'] = df['Yeni Metin'].str.replace('\d','')
#stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('turkish')
df['Yeni Metin'] = df['Yeni Metin'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

In [5]:
df[['Metin',"Yeni Metin"]][:10]

,Metin,Yeni Metin
0,evet anlatıldığı gibi,evet anlatıldığı
1,Daha öncede almıştım bu cihazdan ense ve sakal...,öncede almıştım cihazdan ense sakal tüketmek o...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,ürün gayet başarılı sakal kesmede başlık sayıs...
3,Daha öncede aynısını almıştım çok güzel ve kal...,öncede aynısını almıştım güzel kaliteli bir ürün
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,erkek kuaförüyüm ense sıfır sakal traşı uygun ...
5,ürün gerçekten çok güzel,ürün gerçekten güzel
6,Ürün beklediğimden güzel çıktı gayet kullanışl...,ürün beklediğimden güzel çıktı gayet kullanışl...
7,güzel makina tavsiye ederim,güzel makina tavsiye ederim
8,tavsiye edebileceğim çok güzel bir makina,tavsiye edebileceğim güzel bir makina
9,ürün geldiğinde şarjı vardı. ilk lullanım önce...,ürün geldiğinde şarjı vardı ilk lullanım önces...


<a id="4"></a><br>
# Kelime Kökleme

In [6]:
import snowballstemmer

stemmer = snowballstemmer.stemmer('turkish')


In [7]:
print(stemmer.stemWords("teşekkürler".split()))

['teşekkür']


In [8]:
df["Kök Metin"]=df["Yeni Metin"].apply(lambda x: " ".join([stemmer.stemWord(word) for word in x.split()]))

In [9]:
df[["Yeni Metin",'Kök Metin']][:10]

,Yeni Metin,Kök Metin
0,evet anlatıldığı,evet anlatıldık
1,öncede almıştım cihazdan ense sakal tüketmek o...,önce almış cihaz en sakal tüketmek on numar sı...
2,ürün gayet başarılı sakal kesmede başlık sayıs...,ür gayet başarıl sakal kesme başlık sayıs bira...
3,öncede aynısını almıştım güzel kaliteli bir ürün,önce aynı almış güzel kalitel bir ür
4,erkek kuaförüyüm ense sıfır sakal traşı uygun ...,erkek kuaför en sıfır sakal traş uygu bir ür
5,ürün gerçekten güzel,ür gerçek güzel
6,ürün beklediğimden güzel çıktı gayet kullanışl...,ür bekledik güzel çık gayet kullanışlı tark bi...
7,güzel makina tavsiye ederim,güzel mak tavsi eder
8,tavsiye edebileceğim güzel bir makina,tavsi edebilecek güzel bir mak
9,ürün geldiğinde şarjı vardı ilk lullanım önces...,ür geldik şarjı var ilk lulla önces saat kadar...


<a id="5"></a><br>
# TF-IDF

In [10]:
tf = (df["Kök Metin"]).apply(lambda x: 
                             pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()

In [11]:
tf

,index,0
0,evet,115.0
1,anlatıldık,66.0
2,önce,30.0
3,almış,331.0
4,cihaz,134.0
...,...,...
12152,anpulde,1.0
12153,gwliyo,1.0
12154,kapattigi,1.0
12155,ayari,1.0


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()
Xtf = vectorizer.fit_transform(df["Kök Metin"])

tokens = vectorizer.get_feature_names()

tf_idf = pd.DataFrame(data = Xtf.toarray(), columns = tokens)

print(tf_idf)

        aa  abajur  abartma  abartmıyorumbu  abartıl  abartılacak  abartıldık  \
0      0.0     0.0      0.0             0.0      0.0          0.0         0.0   
1      0.0     0.0      0.0             0.0      0.0          0.0         0.0   
2      0.0     0.0      0.0             0.0      0.0          0.0         0.0   
3      0.0     0.0      0.0             0.0      0.0          0.0         0.0   
4      0.0     0.0      0.0             0.0      0.0          0.0         0.0   
...    ...     ...      ...             ...      ...          ...         ...   
15165  0.0     0.0      0.0             0.0      0.0          0.0         0.0   
15166  0.0     0.0      0.0             0.0      0.0          0.0         0.0   
15167  0.0     0.0      0.0             0.0      0.0          0.0         0.0   
15168  0.0     0.0      0.0             0.0      0.0          0.0         0.0   
15169  0.0     0.0      0.0             0.0      0.0          0.0         0.0   

       abi  abilir  abiç  .

<a id="6"></a><br>
# Duygu Analizi

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score 
from sklearn import model_selection
from sklearn import naive_bayes

In [14]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(tf_idf,
                                                                   df["Durum"],
                                                                    test_size=0.30,
                                                                    random_state = 42
                                                                    )

In [15]:
nb_multi = naive_bayes.MultinomialNB()
nb_model_multi = nb_multi.fit(train_x,train_y)
y_pred_nb_multi=nb_model_multi.predict(test_x)
print("Multinominal Accuracy:", accuracy_score(test_y, y_pred_nb_multi, normalize=True))
print(classification_report(test_y, y_pred_nb_multi))

y_pred_nb_multi_train=nb_model_multi.predict(train_x)
print("Multinominal train Accuracy:", accuracy_score(train_y, y_pred_nb_multi_train, normalize=True))
print(classification_report(train_y, y_pred_nb_multi_train))

print("------------------------------------------------------------------------------------------")

print("test cross val skor:" , cross_val_score(nb_model_multi, test_x, test_y, cv = 10).mean())

print("train cross val skor:" ,cross_val_score(nb_model_multi, train_x, train_y, cv = 10).mean())

Multinominal Accuracy: 0.8635464733025708
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      2065
           1       0.85      0.95      0.90      2051
           2       1.00      0.03      0.06       435

    accuracy                           0.86      4551
   macro avg       0.91      0.64      0.62      4551
weighted avg       0.88      0.86      0.82      4551

Multinominal train Accuracy: 0.8857707882098126
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      4913
           1       0.86      0.97      0.92      4748
           2       1.00      0.05      0.09       958

    accuracy                           0.89     10619
   macro avg       0.92      0.66      0.65     10619
weighted avg       0.90      0.89      0.85     10619

------------------------------------------------------------------------------------------
test cross val skor: 0.8549754193175246
train cross val s

<a id="7"></a><br>
# Yeni Gelen Yorum Duygusu Tahmini

In [16]:
yeni_yorum = pd.Series("168 boyum 83 kg yum gerçekten üzerime şahane oldu ve çok beğendim sadece yaka kısmı bana göre biraz açık onada iğne yada gizli bir dikişle halledicem  kalitesi kumaşı herseyiyle çok güzel teşekkürler") #olumlu

yeni_yorum2 = pd.Series("Yanlış elbise göndermişler. Bu elbiseyle alakası yok") #olumsuz

yeni_yorum3 = pd.Series("az kalın olabilirdi") #nötr

In [17]:
yeni_yorum = vectorizer.transform(yeni_yorum)
yeni_yorum

<1x12126 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [23]:
nb_model_multi.predict(yeni_yorum)

array([1])

In [19]:
yeni_yorum2 = vectorizer.transform(yeni_yorum2)
yeni_yorum2

<1x12126 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [20]:
nb_model_multi.predict(yeni_yorum2)

array([0])

In [21]:
yeni_yorum3 = vectorizer.transform(yeni_yorum3)
yeni_yorum3

<1x12126 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [22]:
nb_model_multi.predict(yeni_yorum3)

array([0])